# Exploratory Data Analysis (EDA)

This notebook conducts exploratory data analysis on the harmonized and model-ready emergency incident datasets for Toronto and New York City. The objective of this analysis is to examine the distribution and variability of emergency response times, identify temporal and operational patterns associated with peak demand and delayed responses, and assess service-level performance beyond simple averages. Particular attention is given to tail delays and response-time threshold breaches, which are critical for understanding operational risk in emergency response systems. The findings from this EDA are used to guide feature engineering, model selection, and comparative analysis in subsequent stages of the project.

